### PubMed5

In [1]:
import numpy as np
from scipy.io import loadmat
import pandas as pd
from DCC_data_utils import make_tfidf, make_PPMI
import random

n_top_terms = 30
n_top_frequent = 1000
n_runs = 100


In [ ]:
#PubMed10_dw = loadmat("/home/saffeldt/Projects/Projects_clustering/DCC/DataDCC/PubMed10/docWordMat.mat")
#doc_term_counts = PubMed10_dw["docWordMat"]
#PubMed10_lb = loadmat("/home/saffeldt/Projects/Projects_clustering/DCC/DataDCC/PubMed10/label.mat")
#labels = PubMed10_lb["label"]
#labels = [item for sublist in labels for item in sublist]
## Load the words and get the freq
#pm10_terms = loadmat("/home/saffeldt/Projects/Projects_NER/Severine/Datasets/10Pubmed/wordList.mat")
###pm10_terms
#
##print(type(pm10_terms["wordList"]))
##print(pm10_terms["wordList"].shape)
##print(np.where(pm10_terms["wordList"]=="wa"))
##print(pm10_terms["wordList"][21963, 0])
##
##print((np.array(doc_term_counts[:, 21963]).all()>0.0))

In [2]:
# Load PubMed original data
PubMed10_dw = loadmat("/home/saffeldt/Projects/Projects_clustering/DCC/DataDCC/PubMed10/docWordMat.mat")
doc_term_counts = PubMed10_dw["docWordMat"]
PubMed10_lb = loadmat("/home/saffeldt/Projects/Projects_clustering/DCC/DataDCC/PubMed10/label.mat")
labels = PubMed10_lb["label"]
labels = [item for sublist in labels for item in sublist]
# Load the words and get the freq
pm10_terms = loadmat("/home/saffeldt/Projects/Projects_NER/Severine/Datasets/10Pubmed/wordList.mat")
##pm10_terms
pm10_terms = pm10_terms["wordList"]
pm10_terms_list = list()

for i in range(pm10_terms.shape[0]):
    pm10_terms_list.append(pm10_terms[i,0].tolist()[0])

pm10_terms = pm10_terms_list

In [3]:
# Extract the PubMed5 data and labels from PubMed10
# ------
#  (6) Hay Fever --> nDoc = 1517
#  (7) Kidney Calculi --> nDoc = 1549
#  (8) Age-related Macular Degeneration --> nDoc = 3283
#  (9) Migraine --> nDoc = 3703
# (10) Otitis --> nDoc = 2596
PubMed5_lb_unique = [6, 7, 8, 9, 10]

PubMed5_doc_idx = list()
for searchval in PubMed5_lb_unique:
    PubMed5_doc_idx.extend(np.where(np.asarray(labels) == searchval)[0])

# Reduce the labels list
labels = [labels[i] for i in PubMed5_doc_idx]

# Reduce the doc_term_counts matrix with the list of 
# document index
mat = doc_term_counts[np.asarray(PubMed5_doc_idx), :]
del doc_term_counts

# Reduce the mat matrix by removing columns of 0
word_org_idx = np.unique(mat.nonzero()[1])
mat = mat[:, word_org_idx]
mat.shape


# Apply the same permutations of rows as done before using RDCC
tmp_perm = np.random.RandomState(seed=42).permutation(mat.shape[0])
mat = mat[tmp_perm,:]
labels = [labels[i] for i in tmp_perm.tolist()]

# Make the TF-IDF matrix
my_norm = 'l2'
s_idf = True
s_tf = False
mat_tfidf = make_tfidf(mat, data_name = "PubMed5", norm = my_norm,
                       smooth_idf = s_idf, sublinear_tf = s_tf, verbose = True)

# Make the word PPMI matrix
isCol = True
mat_ppmi = make_PPMI(mat_tfidf, isCol = isCol, verbose = True)


# ---------------
# TF-IDF transform
# ---------------
# X_tfidf shape: (12648, 19518)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

# X_tfidf min/max: 0.0, 0.9369476151509215
# X_tfidf sparsity: 0.9967840670144149
# ---------------
(19518, 12648)
(12648, 19518)
(19518, 1)
(1, 19518)
--1-- 10681588
--2-- 7730842
# ---------------
# SPPMI transform from X_TFIDF
# ---------------
# SPPMI shape: (19518, 19518)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

# SPPMI min/max: 0.0, 11.5613
# SPPMI sparsity: 0.9797
# ---------------


### Compute a reference PPMI mean value (random)

In [63]:
all_rand_mean_ppmi = list()

# Get randomly n_top_terms words among the n_top_frequent terms
terms_freq = (mat.sum(axis=0))
# Sort by decreasing count sum
terms_freq_order = np.array((-terms_freq).argsort()).flatten().tolist()

for iRun in range(n_runs):
    rand_idx = random.sample(range(0, n_top_frequent), n_top_terms)
    terms_freq_ref = [terms_freq_order[i] for i in rand_idx]
    #print(terms_freq_ref)

    # Get the corresponding TF-IDF vector
    #mat_tfidf_mostFreq = mat_tfidf[:,terms_freq_order[:n_terms]]
    mat_ppmi_mostFreq = mat_ppmi[:,terms_freq_ref]
    mat_ppmi_mostFreq = mat_ppmi_mostFreq[terms_freq_ref, :]
    #print(mat_ppmi_mostFreq.shape)

    # Compute the mean PPMI
    #print(mat_ppmi_mostFreq)
    tmp_list = []
    for i in range(mat_ppmi_mostFreq.shape[0]):
        for j in range(i+1, mat_ppmi_mostFreq.shape[1]):
            tmp_list.append(mat_ppmi_mostFreq[i,j])
    tmp_mean = np.mean(tmp_list)
    all_rand_mean_ppmi.append(tmp_mean)
    
print("Median PPMI", np.median(all_rand_mean_ppmi))
print("Max PPMI", np.max(all_rand_mean_ppmi))
print("Min PPMI", np.min(all_rand_mean_ppmi))
print("\nMean PPMI", np.mean(all_rand_mean_ppmi))
print("--> PPMI std", np.std(all_rand_mean_ppmi))

Median PPMI 0.17638101553277746
Max PPMI 0.3876594252223152
Min PPMI 0.07937405019837619

Mean PPMI 0.18694311546992676
--> PPMI std 0.06030022999096707


In [64]:
import os
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score

out_dir = "/home/saffeldt/Projects/Projects_clustering/DCC/output"
data_name = "PubMed5"

version = "mulPPMI_ColW-RowZ" # mulPPMI_Col, mulPPMI_Row, mulPPMI_ColZ-RowW, mulPPMI_ColW-RowZ
all_versions = ["mulPPMI_Col", "mulPPMI_Row", "mulPPMI_ColZ-RowW", "mulPPMI_ColW-RowZ", "id"]
p = 0
n_init = 10
n_runs = 20
    
all_top_terms = dict()
all_top_terms_idx = dict()

for version in all_versions:
    print("# Version", version)
    all_top_terms[version] = list()
    all_top_terms_idx[version] = list()
    
    if version == "id":
        p = 0
        
    # Load the column partition
    col_part_path = os.path.join(out_dir, data_name, "{}init_{}runs_p{}_{}".format(n_init, n_runs, p, version), "dcc_l2_sidfTrue_stfFalse_k5_stoch70_init10_iter100__axis1.npy")
    col_part = np.load(col_part_path)

    # Load the row partition
    row_part_path = os.path.join(out_dir, data_name, "{}init_{}runs_p{}_{}".format(n_init, n_runs, p, version), "dcc_l2_sidfTrue_stfFalse_k5_stoch70_init10_iter100__axis0.npy")
    row_part = np.load(row_part_path)

    # Select one of the co-clustering result
    part_num = -1
    best_nmi = 0.0

    # Evaluate the row partition and select the partition with the best NMI
    for i_part in range(n_runs):
        ari = adjusted_rand_score(labels, row_part[i_part,:])
        nmi = normalized_mutual_info_score(labels, row_part[i_part,:], average_method="arithmetic")
        #print("# --> ({}) NMI".format(i_part), "{0:.4f}".format(nmi), 
        #      "| ({}) ARI".format(i_part), "{0:.4f}".format(ari), "\n")

        if nmi > best_nmi:
            part_num =  i_part
            best_nmi = nmi

    print("{} Best row partition: {} | ".format(version, part_num), "NMI {0:0.4f}".format(best_nmi))

    # All series of words
    all_series = list()
    nbr_clust = len(np.unique(col_part))

    for n_clust in range(nbr_clust):
        # Get the col and row indices corresponding to the current cluster
        index_clust_col = np.where(col_part[part_num,:] == n_clust)
        index_clust_col = index_clust_col[0].flatten().tolist()
        index_clust_row = np.where(row_part[part_num,:] == n_clust)
        index_clust_row = index_clust_row[0].flatten().tolist()
        
        # Extract the co-cluster and compute the count sum for all words of the co-cluster
        curr_doc_term = mat[index_clust_row,:]
        curr_doc_term = curr_doc_term[:,index_clust_col]
        tmp_terms_freq = (curr_doc_term.sum(axis=0))
        print("n_clust", n_clust)
        #print(curr_doc_term)

        # Sort by decreasing count sum
        tmp_order = (-tmp_terms_freq).argsort()
        tmp_order = np.array(tmp_order[0]).flatten().tolist()
        # Convert the ordered col index of the co-cluster into the original matrix index
        tmp_order_org_idx = [index_clust_col[i] for i in tmp_order]
        
        curr_ppmi = mat_ppmi[:,tmp_order_org_idx[:n_top_terms]]
        curr_ppmi = curr_ppmi[tmp_order_org_idx[:n_top_terms],:]
        all_top_terms_idx[version].append(tmp_order_org_idx[:n_top_terms])
        tmp_list = []
        for i in range(curr_ppmi.shape[0]):
            for j in range(i+1, curr_ppmi.shape[1]):
                tmp_list.append(curr_ppmi[i,j])
        tmp_mean = np.mean(tmp_list)
        print("-->> PPMI:", tmp_mean)
        
        # Convert to original index to get the words
        tmp_org_idx = (word_org_idx[index_clust_col]).tolist()
        tmp_terms = [pm10_terms[i] for i in tmp_org_idx]
        top_terms = [tmp_terms[i] for i in tmp_order[:n_top_terms]]    

        # See the top terms
        top_terms = np.stack(top_terms, axis = 0 )
        print("-->", top_terms, "\n")
        all_series.append(pd.Series(top_terms))

    print(pd.concat([all_series[0], all_series[1], all_series[2], all_series[3], all_series[4]], axis=1))
    print("\n")
    #import matplotlib.pyplot as plt
    #plt.plot(tmp_terms_freq[:,tmp_order][0][:,:500])
    #plt.show()

# Version mulPPMI_Col
mulPPMI_Col Best row partition: 11 |  NMI 0.9126
n_clust 0
-->> PPMI: 1.153728732673218
--> ['stone' 'wa' 'patient' 'renal' 'calcium' 'urinari' 'kidnei' 'oxal' 'rate'
 'urin'] 

n_clust 1
-->> PPMI: 0.33078413430564346
--> ['migrain' 'headach' 'thi' 'treatment' 'ar' 'pain' 'clinic' 'attack'
 'than' 'mai'] 

n_clust 2
-->> PPMI: 1.367268014385637
--> ['otiti' 'ear' 'children' 'media' 'middl' 'acut' 'antibiot' 'aom' 'tube'
 'om'] 

n_clust 3
-->> PPMI: 1.2517673125457525
--> ['allerg' 'nasal' 'rhiniti' 'symptom' 'pollen' 'studi' 'season' 'effect'
 'allergen' 'asthma'] 

n_clust 4
-->> PPMI: 0.8861379874902059
--> ['macular' 'ey' 'visual' 'amd' 'retin' 'acuiti' 'associ' 'degen'
 'neovascular' 'signif'] 

         0          1         2         3            4
0    stone    migrain     otiti    allerg      macular
1       wa    headach       ear     nasal           ey
2  patient        thi  children   rhiniti       visual
3    renal  treatment     media   symptom      

In [7]:
version = "mulPPMI_ColZ-RowW"
#version = "mulPPMI_Row"

for i in range(5):
    print("\n# Clust", i)
        
    # PPMI
    # ----
    list_mean = list()
    for j in range(5):
        curr_ppmi = mat_ppmi[:,all_top_terms_idx[version][i]]
        curr_ppmi = curr_ppmi[all_top_terms_idx[version][j],:]
        #tmp_mean = (np.triu(curr_ppmi.todense(), 1).sum())/(curr_ppmi.shape[0]*(curr_ppmi.shape[0]-1)/2)
        tmp_list = []
        for k in range(curr_ppmi.shape[0]):
            for l in range(k+1, curr_ppmi.shape[1]):
                tmp_list.append(curr_ppmi[k,l])
        tmp_mean = np.mean(tmp_list)
        if j != i: 
            list_mean.append(tmp_mean)
        else:
            print("-->> self PPMI:", tmp_mean)
    print("-->> versus all PPMI:", np.mean(list_mean))


# Clust 0
-->> self PPMI: 0.5045375910715757
-->> versus all PPMI: 0.06883567420493292

# Clust 1
-->> self PPMI: 0.8857582711858465
-->> versus all PPMI: 0.07286667036349084

# Clust 2
-->> self PPMI: 1.2114945964468498
-->> versus all PPMI: 0.059344202280147675

# Clust 3
-->> self PPMI: 0.8505059313733708
-->> versus all PPMI: 0.09321821802968558

# Clust 4
-->> self PPMI: 0.31800916790479783
-->> versus all PPMI: 0.08487010797665331


## NPMI within

In [4]:
n_top_terms=30

In [5]:
import numpy as np
import subprocess
import re
import os

jar_path = '/home/saffeldt/Projects/Projects_clustering/DCC/Palmetto/palmetto-0.1.0-jar-with-dependencies.jar'
wiki_file = '/home/saffeldt/Projects/Projects_clustering/DCC/Palmetto/Wikipedia_bd/wikipedia_bd'
c_measure = 'npmi'
topic_file = '/home/saffeldt/Projects/Projects_clustering/DCC/Palmetto/topic_files/current_topics.txt'
out_file = '/home/saffeldt/Projects/Projects_clustering/DCC/Palmetto/topic_files/current_topics_out.txt'

# PUBMED5 top terms
word_list = ['wa', 'macular', 'eye', 'visual', 'amd', 'retinal', 'acuity', 'associate', 'degeneration', 'group']
word_list = ['otitis', 'ear', 'child', 'media', 'middle', 'acute', 'antibiotics', 'aom', 'tube', 'om']
word_list = ['stone', 'renal', 'calcium', 'urinary', 'kidney', 'oxalate', 'rate', 'urine', 'percutaneous', 'calculi']
word_list = ['allergy', 'nasal', 'rhinitis', 'symptom', 'pollen', 'seasonal', 'effect', 'allergens', 'asthma', 'increase']
word_list = ['migraine', 'patient', 'headache', 'study', 'thi', 'treatment', 'ar', 'pain', 'attack', 'associate']

# AMD vs OTITIS
word_list = ["wa", "macular", "eye", "visual", "amd", "retinal", "acuity", "associate", "degeneration", 
             "group", "neovascular", "significant", "month", "diabetic", "no", "improve", "risk", 
             "choroiditis", "edema", "measure", "intravitreal", "result", "thick", "change", "injection", 
             "case", "evaluate", "development", "show", "factor"]
word_list = ["otitis","ear","children","media","middle","acute","antibiotics","aom","tube","om","ag",
             "hear","effusion","year","chronic","dai","infection","present","cause","isolate",
             "recurrent","pneumonia","bacterial","found","tympan","vaccin","mastoiditis","complication",
             "pneumococcal","influenza"]
word_list = ["stone","renal","calcium","urinary","kidney","oxalate","rate","urine", "percutaneous",
             "calculi","crystal","lower","shock","lithotripsy","wave", "fragment","ureter","acid",
             "require","excretion","nephrolithotomy","procedure", "formation","protein","material",
             "tract","management","metabolite","uric","pcnl"]
word_list = ["allergic", "nasal", "rhinitis", "symptom", "pollen", "seasonal", "effect", "allergen", 
             "asthma", "increase", "significantly", "subject", "score", "level", "cell", "ig", 
             "placebo", "allergy", "test", "sensitive", "specific", "total", "immunotherapy", 
             "active", "grass", "eosinophil", "intranasal", "reduce", "challenge", "daily"]
word_list = ["migraine", "patient", "headache", "study", "thi", "treatment", "ar", "pain", "clinical", 
             "compare", "attack", "than", "may", "different", "dure", "aura", "report", "control", 
             "severe", "ha", "mg", "medical", "response", "efficacy", "sumatriptan", "assess", 
             "include", "data", "triptan", "drug"]

word_list = ["macular", "degeneration", "retinal", "edema", "diabetic", "acuity", "visual", "amd", 
                  "injection", "eye", "neovascular", "risk", "factor", "intravitreal", "significant", 
                  "measure", "associate", "evaluate", "improve", "result", "change", "development", 
                  "case", "thick", "show", "choroiditis", "group", "month", "no", "wa"]

n_words = len(word_list)
n_pairs = int((n_words*(n_words-1)/2))

all_npmi = np.zeros((n_words, n_words))

# Call Palmetto/NPMI
# ----
cmd = "java -jar '{}' '{}' '{}' '{}' > '{}'".format(jar_path, wiki_file, c_measure, topic_file, out_file)
for wi in range(n_words-1):
    # Write all word pairs in a topic file
    outF = open(topic_file, "w")
    for wj in range(wi+1, n_words):
        line = '{} {}\n'.format(word_list[wi], word_list[wj])
        outF.write(line)
    outF.close()
    
    # Call Palmetto for NPMI
    result = os.system(cmd)

    if result == 0:
        # Read out file line by line
        file1 = open(out_file, 'r') 
        #next(file1)
        Lines = file1.readlines()[1:]
        
        # Strips the newline character
        for line in Lines: 
            tmp_list = line.strip().split()
            #print(tmp_list)
            tmp_value = tmp_list[1]
            tmp_wi_str = (tmp_list[2])[1:-1]
            tmp_wj_str = (tmp_list[3])[0:-1]
            tmp_wi_idx = word_list.index(tmp_wi_str)
            tmp_wj_idx = word_list.index(tmp_wj_str)
            all_npmi[tmp_wi_idx, tmp_wj_idx] = tmp_value
            all_npmi[tmp_wj_idx, tmp_wi_idx] = all_npmi[tmp_wi_idx, tmp_wj_idx]
            #print(tmp_wi_str, tmp_wj_str, tmp_value)
        
#print(all_npmi)
# Compute the NPMI3 scores
k = 5
all_npmi_argsort = np.argsort(-all_npmi, axis = 0)
all_npmi_score = []

for i_word in range(len(word_list)):
    tmp_order_idx = all_npmi_argsort[:,i_word].flatten().tolist()
    tmp_order_idx = [idx for idx in tmp_order_idx if idx != i_word ]
    tmp_npmi_score = all_npmi[tmp_order_idx[:k], i_word]
    print(i_word, word_list[i_word], "--> ", [word_list[i] for i in tmp_order_idx[:k]], tmp_npmi_score)
    all_npmi_score.append(np.mean(tmp_npmi_score))

import pandas as pd

word_npmi_order_idx = np.argsort(-np.array(all_npmi_score))
word_npmi_order_str_series = pd.Series([word_list[i] for i in word_npmi_order_idx])
word_npmi_order_val_series = pd.Series([all_npmi_score[i] for i in word_npmi_order_idx])
pd.concat([word_npmi_order_str_series, word_npmi_order_val_series], axis = 1)

0 macular -->  ['degeneration', 'neovascular', 'edema', 'retinal', 'diabetic'] [0.80604 0.62993 0.58788 0.53307 0.48512]
1 degeneration -->  ['macular', 'retinal', 'amd', 'diabetic', 'eye'] [0.80604 0.55617 0.4049  0.37944 0.19901]
2 retinal -->  ['degeneration', 'macular', 'edema', 'acuity', 'diabetic'] [0.55617 0.53307 0.42495 0.39092 0.38384]
3 edema -->  ['macular', 'retinal', 'diabetic', 'injection', 'associate'] [0.58788 0.42495 0.36009 0.21387 0.15407]
4 diabetic -->  ['macular', 'retinal', 'degeneration', 'edema', 'injection'] [0.48512 0.38384 0.37944 0.36009 0.22956]
5 acuity -->  ['visual', 'retinal', 'eye', 'measure', 'improve'] [0.54285 0.39092 0.29803 0.20897 0.18933]
6 visual -->  ['acuity', 'retinal', 'macular', 'eye', 'degeneration'] [0.54285 0.29586 0.23859 0.17327 0.13277]
7 amd -->  ['macular', 'degeneration', 'risk', 'factor', 'development'] [0.47831 0.4049  0.07705 0.05191 0.04979]
8 injection -->  ['intravitreal', 'diabetic', 'edema', 'improve', 'risk'] [0.51483 0

,0,1
0,macular,0.608408
1,degeneration,0.469112
2,retinal,0.457790
3,diabetic,0.367610
4,edema,0.348172
5,acuity,0.326020
6,visual,0.276668
7,eye,0.250324
8,injection,0.233062
9,risk,0.227270


In [6]:
# Save all NPMI values as matrix
np.savetxt("/home/saffeldt/Projects/Projects_clustering/DCC/output/PubMed5/NPMI/allNPMI_30w_k5_amd.csv", all_npmi, 
              delimiter = ",")

In [7]:
# Compute all pairwise average
all_npmi_mean = (np.triu(all_npmi, 1).sum())/(all_npmi.shape[0]*(all_npmi.shape[0]-1)/2)
print("# NPMI average: ", all_npmi_mean)

# NPMI average:  -0.05100016091954022


## NPMIk between

In [57]:
import numpy as np
import subprocess
import re

jar_path = '/home/saffeldt/Projects/Projects_clustering/DCC/Palmetto/palmetto-0.1.0-jar-with-dependencies.jar'
wiki_file = '/home/saffeldt/Projects/Projects_clustering/DCC/Palmetto/Wikipedia_bd/wikipedia_bd'
c_measure = 'npmi'
topic_file = '/home/saffeldt/Projects/Projects_clustering/DCC/Palmetto/topic_files/current_topics.txt'
out_file = '/home/saffeldt/Projects/Projects_clustering/DCC/Palmetto/topic_files/current_topics_out.txt'

# PUBMED5 frequency top terms
## -- 10 -- ##
#word_list_amd = ['wa', 'macular', 'eye', 'visual', 'amd', 'retinal', 'acuity', 'associate', 'degeneration', 'group']
##word_list_otitis = ['otitis', 'ear', 'children', 'media', 'middle', 'acute', 'antibiotics', 'aom', 'tube', 'om']
#word_list_otitis = ['otitis', 'ear', 'child', 'media', 'middle', 'acute', 'antibiotics', 'aom', 'tube', 'om']
#word_list_kidney = ['stone', 'renal', 'calcium', 'urinary', 'kidney', 'oxalate', 'rate', 'urine', 'percutaneous', 'calculi']
#word_list_hayfever = ['allergy', 'nasal', 'rhinitis', 'symptom', 'pollen', 'seasonal', 'effect', 'allergens', 'asthma', 'increase']
#word_list_migraine = ['migraine', 'patient', 'headache', 'study', 'thi', 'treatment', 'ar', 'pain', 'attack', 'associate']

# PUBMED5 NPMIk top terms
## -- 30 -- ##
word_list_amd = ["macular", "degeneration", "retinal", "edema", "diabetic", "acuity", "visual", "amd", 
                  "injection", "eye", "neovascular", "risk", "factor", "intravitreal", "significant", 
                  "measure", "associate", "evaluate", "improve", "result", "change", "development", 
                  "case", "thick", "show", "choroiditis", "group", "month", "no", "wa"]
word_list_otitis = ["otitis", "infection", "pneumonia", "bacterial", "acute", "chronic", "antibiotics", 
                    "recurrent", "effusion", "complication", "influenza", "ear", "cause", "pneumococcal", 
                    "isolate", "media", "tube", "hear", "middle", "vaccin", "present", "found", "tympan", 
                    "mastoiditis", "children", "ag", "year", "aom", "dai", "om"]
word_list_kidney = ["urinary", "excretion", "ureter", "urine", "kidney", "renal", "uric", "oxalate", 
                    "acid", "metabolite", "calcium", "lithotripsy", "tract", "calculi", "protein", 
                    "crystal", "stone", "percutaneous", "procedure", "shock", "wave", "lower", 
                    "formation", "nephrolithotomy", "rate", "fragment", "require", "material", 
                    "management", "pcnl"]
word_list_hayfever = ["allergic", "rhinitis", "allergy", "asthma", "allergen", "immunotherapy", "pollen", "nasal", 
                      "symptom", "eosinophil", "cell", "seasonal", "effect", "sensitive", "placebo", "reduce", 
                      "increase", "test", "specific", "grass", "level", "subject", "total", "active", "daily", 
                      "intranasal", "score", "challenge", "ig", "significantly"]
word_list_migraine = ["migraine", "headache", "triptan", "treatment", "pain", "patient", "efficacy", 
                      "clinical", "drug", "severe", "medical", "sumatriptan", "aura", "assess", "study", 
                      "compare", "data", "mg", "response", "attack", "control", "report", "may", 
                      "different", "ha", "include", "thi", "ar", "than", "dure"]


# One vs Others,
word_list = word_list_migraine
word_list.extend(word_list_hayfever)

n_words = len(word_list)
n_pairs = int((n_words*(n_words-1)/2))

all_npmi = np.zeros((n_words, n_words))

# Call Palmetto/NPMI
# ----
cmd = "java -jar '{}' '{}' '{}' '{}' > '{}'".format(jar_path, wiki_file, c_measure, topic_file, out_file)
for wi in range(n_words-1):
    print("# word", wi)
    # Write all word pairs in a topic file
    outF = open(topic_file, "w")
    for wj in range(wi+1, n_words):
        line = '{} {}\n'.format(word_list[wi], word_list[wj])
        outF.write(line)
    outF.close()
    
    # Call Palmetto for NPMI
    result = os.system(cmd)

    if result == 0:
        # Read out file line by line
        file1 = open(out_file, 'r') 
        #next(file1)
        Lines = file1.readlines()[1:]
        
        # Strips the newline character
        for line in Lines: 
            tmp_list = line.strip().split()
            #print(tmp_list)
            tmp_value = tmp_list[1]
            tmp_wi_str = (tmp_list[2])[1:-1]
            tmp_wj_str = (tmp_list[3])[0:-1]
            tmp_wi_idx = word_list.index(tmp_wi_str)
            tmp_wj_idx = word_list.index(tmp_wj_str)
            all_npmi[tmp_wi_idx, tmp_wj_idx] = tmp_value
            all_npmi[tmp_wj_idx, tmp_wi_idx] = all_npmi[tmp_wi_idx, tmp_wj_idx]
            #print(tmp_wi_str, tmp_wj_str, tmp_value)
        

# word 0
# word 1
# word 2
# word 3
# word 4
# word 5
# word 6
# word 7
# word 8
# word 9
# word 10
# word 11
# word 12
# word 13
# word 14
# word 15
# word 16
# word 17
# word 18
# word 19
# word 20
# word 21
# word 22
# word 23
# word 24
# word 25
# word 26
# word 27
# word 28
# word 29
# word 30
# word 31
# word 32
# word 33
# word 34
# word 35
# word 36
# word 37
# word 38
# word 39
# word 40
# word 41
# word 42
# word 43
# word 44
# word 45
# word 46
# word 47
# word 48
# word 49
# word 50
# word 51
# word 52
# word 53
# word 54
# word 55
# word 56
# word 57
# word 58


In [58]:
#print(all_npmi)
# Compute the NPMIk scores
k = 5
#print(word_list)
sub_npmi = all_npmi[n_top_terms:(2*n_top_terms),:n_top_terms]
sub_npmi_argsort = np.argsort(-sub_npmi, axis = 0)
sub_npmi_score = []

for i_word in range(sub_npmi.shape[1]):
    tmp_order_idx = sub_npmi_argsort[:,i_word].flatten().tolist()
    tmp_npmi_score = sub_npmi[tmp_order_idx[:k], i_word]
    print(i_word, word_list[i_word], "--> ", [word_list[i+n_top_terms] for i in tmp_order_idx[:k]], 
          tmp_npmi_score)
    sub_npmi_score.append(np.mean(tmp_npmi_score))

import pandas as pd
word_npmi_str_series = pd.Series(word_list[:n_top_terms])
word_npmi_val_series = pd.Series(sub_npmi_score)
pd.concat([word_npmi_str_series, word_npmi_val_series], axis = 1)

0 migraine -->  ['symptom', 'reduce', 'effect', 'daily', 'increase'] [0.34245 0.1618  0.14696 0.14375 0.08525]
1 headache -->  ['symptom', 'asthma', 'allergic', 'nasal', 'allergy'] [0.45483 0.29063 0.28529 0.27715 0.25447]
2 triptan -->  ['significantly', 'intranasal', 'rhinitis', 'eosinophil', 'immunotherapy'] [ 0.      -0.01509 -0.04442 -0.04502 -0.0554 ]
3 treatment -->  ['immunotherapy', 'symptom', 'placebo', 'asthma', 'rhinitis'] [0.31755 0.3135  0.31244 0.28063 0.27758]
4 pain -->  ['symptom', 'placebo', 'allergic', 'asthma', 'reduce'] [0.37391 0.26557 0.18775 0.18225 0.16965]
5 patient -->  ['symptom', 'placebo', 'allergic', 'immunotherapy', 'asthma'] [0.38036 0.3349  0.2809  0.28038 0.27153]
6 efficacy -->  ['placebo', 'immunotherapy', 'symptom', 'effect', 'test'] [0.39342 0.38783 0.20477 0.2006  0.19726]
7 clinical -->  ['symptom', 'allergy', 'placebo', 'immunotherapy', 'asthma'] [0.31454 0.29977 0.2975  0.2907  0.21874]
8 drug -->  ['placebo', 'intranasal', 'allergy', 'effect

,0,1
0,migraine,0.176042
1,headache,0.312474
2,triptan,-0.031986
3,treatment,0.300340
4,pain,0.235826
5,patient,0.309614
6,efficacy,0.276776
7,clinical,0.284250
8,drug,0.227676
9,severe,0.262578


## CosSim within coCluster from doc-term counts

In [ ]:
n_top_terms=5

In [ ]:
import os
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

out_dir = "/home/saffeldt/Projects/Projects_clustering/DCC/output"
data_name = "PubMed5"

version = "mulPPMI_ColW-RowZ" # mulPPMI_Col, mulPPMI_Row, mulPPMI_ColZ-RowW, mulPPMI_ColW-RowZ
#all_versions = ["mulPPMI_Col", "mulPPMI_Row", "mulPPMI_ColZ-RowW", "mulPPMI_ColW-RowZ", "id"]
all_versions = ["mulPPMI_ColZ-RowW"]
p = 1
n_init = 10
n_runs = 20
    
all_top_terms = dict()
all_top_terms_idx = dict()

for version in all_versions:
    print("# Version", version)
    all_top_terms[version] = []
    all_top_terms_idx[version] = []
    
    if version == "id":
        p = 0
        
    # Load the column partition
    col_part_path = os.path.join(out_dir, data_name, "{}init_{}runs_p{}_{}".format(n_init, n_runs, p, version), "dcc_l2_sidfTrue_stfFalse_k5_stoch70_init10_iter100__axis1.npy")
    col_part = np.load(col_part_path)

    # Load the row partition
    row_part_path = os.path.join(out_dir, data_name, "{}init_{}runs_p{}_{}".format(n_init, n_runs, p, version), "dcc_l2_sidfTrue_stfFalse_k5_stoch70_init10_iter100__axis0.npy")
    row_part = np.load(row_part_path)

    # Select one of the co-clustering result
    part_num = -1
    best_nmi = 0.0

    # Evaluate the row partition and select the partition with the best NMI
    for i_part in range(n_runs):
        ari = adjusted_rand_score(labels, row_part[i_part,:])
        nmi = normalized_mutual_info_score(labels, row_part[i_part,:], average_method="arithmetic")
        #print("# --> ({}) NMI".format(i_part), "{0:.4f}".format(nmi), 
        #      "| ({}) ARI".format(i_part), "{0:.4f}".format(ari), "\n")

        if nmi > best_nmi:
            part_num =  i_part
            best_nmi = nmi

    print("{} Best row partition: {} | ".format(version, part_num), "NMI {0:0.4f}".format(best_nmi))

    # All series of words
    all_series = []
    nbr_clust = len(np.unique(col_part))

    for n_clust in range(nbr_clust):
        print("n_clust", n_clust)
        # Get the col and row indices corresponding to the current cluster
        index_clust_col = np.where(col_part[part_num,:] == n_clust)
        index_clust_col = index_clust_col[0].flatten().tolist()
        index_clust_row = np.where(row_part[part_num,:] == n_clust)
        index_clust_row = index_clust_row[0].flatten().tolist()
        
        # Extract the co-cluster
        curr_doc_term = mat[index_clust_row,:]
        curr_doc_term = curr_doc_term[:,index_clust_col]
        print("--> curr_doc_term.shape", curr_doc_term.shape)
        tmp_terms_freq = (curr_doc_term.sum(axis = 0))
        tmp_terms_freq = tmp_terms_freq.flatten().tolist()
        tmp_terms_freq = tmp_terms_freq[0]
        print("--> tmp_terms_freq[:10]", tmp_terms_freq[:10], len(tmp_terms_freq))
        
        # Sort by decreasing count sum
        tmp_order = (-np.array(tmp_terms_freq)).argsort()
        tmp_order = tmp_order.flatten().tolist()
        print("--> tmp_order[:10]", tmp_order[:10], len(tmp_order))
        print("--> tmp freq order[:10]", [tmp_terms_freq[i] for i in tmp_order[:10]])
              
        # Convert the ordered col index of the co-cluster into the original matrix index
        tmp_order_org_idx = [index_clust_col[i] for i in tmp_order]
        
        # Convert to original index to get the words
        tmp_org_idx = (word_org_idx[index_clust_col]).tolist()
        tmp_terms = [pm10_terms[i] for i in tmp_org_idx]
        top_terms = [tmp_terms[i] for i in tmp_order[:n_top_terms]]    

        # See the top terms
        top_terms = np.stack(top_terms, axis = 0 )
        print("-->", top_terms, "\n")
        
        # Compute dot product
        # Reorder doc-term sub mat
        curr_doc_term = curr_doc_term[:,tmp_order]        
        curr_doc_term = normalize(curr_doc_term, norm='l2', axis = 0)       
        Sc = curr_doc_term.T * curr_doc_term
        Sc = Sc[:n_top_terms,:n_top_terms]
        #print("MIN", np.min(np.triu(Sc.todense(), 1)))
        #print(np.triu(Sc.todense(), 1))
        #print(Sc.todense())
        #print("MAX", np.max(np.triu(Sc.todense(), 1)))
        tmp_list = []
        for i in range(n_top_terms):
            for j in range(i+1, n_top_terms):
                if Sc[i,j] > 0.0:
                    tmp_list.append(Sc[i,j])
                    print(top_terms[i], top_terms[j], Sc[i,j])
                
        tmp_mean = np.mean(tmp_list)
        print("-->> Mean CosSim:", tmp_mean, '\n')
                
        
    #print(pd.concat([all_series[0], all_series[1], all_series[2], all_series[3], all_series[4]], axis=1))
    #print("\n")
    #import matplotlib.pyplot as plt
    #plt.plot(tmp_terms_freq[:,tmp_order][0][:,:500])
    #plt.show()